In [ ]:
import cv2
import numpy as np
import pandas as pd
import glob
import os
import shutil

import matplotlib.pyplot as plt
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub

In [ ]:
!cp /content/drive/MyDrive/python_split_image_by_ch.zip .

In [ ]:
!unzip /content/python_split_image_by_ch.zip

Archive:  /content/python_split_image_by_ch.zip
   creating: data/
  inflating: data/00001_b.jpg        
  inflating: data/00001_g.jpg        
  inflating: data/00001_r.jpg        
 extracting: data/00002_b.jpg        
 extracting: data/00002_g.jpg        
 extracting: data/00002_r.jpg        
  inflating: data/00003_b.jpg        
  inflating: data/00003_g.jpg        
  inflating: data/00003_r.jpg        
  inflating: data/00004_b.jpg        
  inflating: data/00004_g.jpg        
  inflating: data/00004_r.jpg        
  inflating: data/00005_b.jpg        
  inflating: data/00005_g.jpg        
  inflating: data/00005_r.jpg        
  inflating: data/00006_b.jpg        
  inflating: data/00006_g.jpg        
  inflating: data/00006_r.jpg        
  inflating: data/00007_b.jpg        
  inflating: data/00007_g.jpg        
  inflating: data/00007_r.jpg        
  inflating: data/00008_b.jpg        
  inflating: data/00008_g.jpg        
  inflating: data/00008_r.jpg        
  inflating: data/000

# Convert images

In [ ]:
def merge_channels(input_dir, output_dir):
    files_list = glob.glob(input_dir+'/*')
    files_list = sorted(files_list)
    files_list_set = set()
    for file in files_list:
        files_list_set.add(file.replace('_b.jpg', '').replace('_g.jpg', '').replace('_r.jpg', ''))
        
    for file in files_list_set:
        file_name = file.replace("\\", "/").split('/')[-1]+'.jpg'
        b = cv2.imread(file+'_b.jpg', cv2.IMREAD_UNCHANGED)
        g = cv2.imread(file+'_g.jpg', cv2.IMREAD_UNCHANGED)
        r = cv2.imread(file+'_r.jpg',cv2.IMREAD_UNCHANGED)

        #     print(b.shape, g.shape, r.shape)

        bgr = cv2.merge([b, g, r])
        cv2.imwrite(output_dir+'/'+file_name, bgr)

In [ ]:
!mkdir  ./output
!mkdir ./output/0
!mkdir ./output/1

mkdir: cannot create directory ‘./output’: File exists


In [ ]:
merge_channels('./data', './output')

In [ ]:
val = pd.read_csv('val.csv', header=None, names=['file', 'cls'])
for i, row in val.iterrows():
  if row['cls'] == False:
    shutil.copy('./output/'+ row['file'], './output/0')
  else:
    shutil.copy('./output/'+ row['file'], './output/1')

# Model and training

In [ ]:
inception_v3 = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"
feature_extractor_model = inception_v3

feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_8 (KerasLayer)  (None, 2048)              21802784  
                                                                 
 dense_11 (Dense)            (None, 128)               262272    
                                                                 
 dense_12 (Dense)            (None, 1)                 129       
                                                                 
Total params: 22,065,185
Trainable params: 262,401
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
  metrics=['acc'])

In [ ]:
data_dir = './output'
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=batch_size)

class_names = train_ds.class_names
class_names

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

Found 100 files belonging to 2 classes.


In [ ]:
NUM_EPOCHS = 30
history = model.fit(train_ds, epochs=NUM_EPOCHS)

Epoch 1/30
4/4 [==============================] - 58s 73ms/step - loss: 40.7670 - acc: 0.5600
Epoch 2/30
4/4 [==============================] - 0s 56ms/step - loss: 36.1858 - acc: 0.6000
Epoch 3/30
4/4 [==============================] - 0s 58ms/step - loss: 27.9889 - acc: 0.5500
Epoch 4/30
4/4 [==============================] - 0s 76ms/step - loss: 15.9086 - acc: 0.6300
Epoch 5/30
4/4 [==============================] - 0s 75ms/step - loss: 7.6759 - acc: 0.6000
Epoch 6/30
4/4 [==============================] - 0s 73ms/step - loss: 7.6860 - acc: 0.6900
Epoch 7/30
4/4 [==============================] - 0s 58ms/step - loss: 4.8492 - acc: 0.7900
Epoch 8/30
4/4 [==============================] - 0s 56ms/step - loss: 7.5943 - acc: 0.6500
Epoch 9/30
4/4 [==============================] - 0s 59ms/step - loss: 5.1950 - acc: 0.7300
Epoch 10/30
4/4 [==============================] - 0s 57ms/step - loss: 5.6314 - acc: 0.7500
Epoch 11/30
4/4 [==============================] - 0s 55ms/step - loss: 3.

In [ ]:
model.save('./saved_model')

INFO:tensorflow:Assets written to: ./saved_model/assets


INFO:tensorflow:Assets written to: ./saved_model/assets


In [ ]:
!zip -r model.zip ./saved_model

updating: saved_model/ (stored 0%)
updating: saved_model/keras_metadata.pb (deflated 84%)
updating: saved_model/variables/ (stored 0%)
updating: saved_model/variables/variables.data-00000-of-00001 (deflated 7%)
updating: saved_model/variables/variables.index (deflated 81%)
updating: saved_model/assets/ (stored 0%)
updating: saved_model/assets.zip (stored 0%)
updating: saved_model/saved_model.pb (deflated 93%)


# Inference

In [ ]:
!rm -r './output/0'
!rm -r './output/1'

In [ ]:
from keras.models import load_model
from keras.preprocessing import image

# model = keras.models.load_model('./saved_model')
input_dir = './output'
file_list = glob.glob(input_dir+"/*")
files = []

for file in file_list:
  file_n = file.replace("\\", "/").split('/')[-1]
  files.append(file_n)

preds = []

for file in file_list:
  img = image.load_img(file, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  pred = model.predict(x)
  pred_b = tf.where(pred > 0.5, 1, 0)
  preds.append(pred_b.numpy().ravel())
preds = np.array(preds).ravel()



In [ ]:
output = pd.DataFrame({'file': files, 'pred': preds})
output['pred'] = output['pred'].astype(bool)
output.to_csv("output.csv", index=False, header=False)

,file,pred
0,00027.jpg,True
1,00055.jpg,True
2,00076.jpg,False
3,00071.jpg,True
4,00013.jpg,False
...,...,...
95,00086.jpg,True
96,00042.jpg,True
97,00040.jpg,False
98,00021.jpg,False


In [ ]:
from keras.models import load_model
from keras.preprocessing import image
import glob

def find_car(input_dir, output_cars ="output.csv"):
  model = keras.models.load_model('./saved_model')
  input_dir = input_dir
  file_list = glob.glob(input_dir+"/*")
  files = []

  for file in file_list:
    file_n = file.replace("\\", "/").split('/')[-1]
    files.append(file_n)

  preds = []

  for file in file_list:
    img = image.load_img(file, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    pred = model.predict(x)
    pred_b = tf.where(pred > 0.5, 1, 0)
    preds.append(pred_b.numpy().ravel())
  preds = np.array(preds).ravel()

  output = pd.DataFrame({'file': files, 'pred': preds})
  output['pred'] = output['pred'].astype(bool)
  output.to_csv(output_cars, index=False, header=False)

In [ ]:
find_car('./output')